<a href="https://colab.research.google.com/github/AmirAsgary/AutoClass-Application-in-Crohn-s-Disease/blob/main/AutoClass_Crohn's_Disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoClass Model
You can simply generate the model using the next cell, and load the weights to it.

In [ ]:
import tensorflow as tf
import pandas as pd
import keras
import numpy as np
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2

# Define the input shape
input_shape = (genes_of_interest_exprs.shape[1],)

# Define the encoder
encoder_input = Input(shape=input_shape)
encoder_output = Dense(64, activation='relu')(encoder_input)
encoder_output = Dense(48, activation='relu')(encoder_output)
encoder_output = Dense(32, activation='relu')(encoder_output)
encoder_output = Dense(16, activation='relu')(encoder_output)
encoder_output = Dense(6, activation = 'relu')(encoder_output)
encoder = Model(encoder_input, encoder_output, name="encoder")

# Define the decoder
decoder_input = Input(shape=(6,))
decoder_output = Dense(16, activation='relu')(decoder_input)
decoder_output = Dense(32, activation='relu')(decoder_output)
decoder_output = Dense(48, activation='relu')(decoder_output)
decoder_output = Dense(64, activation='relu')(decoder_output)
decoder_output = Dense(input_shape[0])(decoder_output)
decoder = Model(decoder_input, decoder_output, name="decoder")

#Define MLP
MLP_input = Input(shape=(6,))
MLP_output = Dense(20, activation="relu", kernel_regularizer=L2(0.001))(MLP_input)
MLP_output = Dense(4, activation="relu")(MLP_output)
MLP_output = Dense(1, activation="sigmoid")(MLP_output)
MLP = Model(MLP_input, MLP_output, name="MLP")


# Define the autoencoder
autoencoder_input = Input(shape=input_shape)
autoencoder_output = decoder(encoder(autoencoder_input))
autoencoder = Model(autoencoder_input, autoencoder_output, name="autoencoder")

#Define Combined Model
encoded_data = encoder(encoder_input)
autoencoder_output = autoencoder(encoder_input)
mlp_output = MLP(encoded_data)
Combined_model = Model(inputs=encoder_input, outputs=[autoencoder_output, mlp_output], name="combined")

# Compile Combined Model
Combined_model.compile(optimizer=Adam(learning_rate=0.005), loss=["mse","binary_crossentropy"], loss_weights=[0.5,0.5])
Combined_model.summary()

In [ ]:
Combined_model.load_weights("/content/Combined_Model.h5")